In [2]:
import pandas as pd
import datetime
import pandas_datareader.data as web
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
from numpy import linalg as la
from sklearn import preprocessing


In [3]:
#Loading the data
start = datetime.datetime(2011, 7, 30)
end = datetime.datetime(2021, 7, 30)
df_SBUX = web.DataReader("SBUX", 'yahoo', start, end)
sbux = df_SBUX['Close'].values
sbux = np.reshape(sbux,(1,-1))
df_MCD = web.DataReader("MCD", 'yahoo', start, end)
mcd = df_MCD['Close'].values
mcd = np.reshape(mcd,(1,-1))


In [12]:
X = np.vstack((sbux,mcd)) 
X1 = np.delete(X, -1, 1) #X_k
X2 = np.delete(X, 0, 1) #X_k+1 (one day)
print(X1.shape)

(2, 2517)


In [9]:
X1_normalized = preprocessing.normalize(X1, norm='l2')
X1_normalized

X2_normalized = preprocessing.normalize(X2, norm='l2')
X2_normalized

print("condition number for X1:", la.cond(X1)) 
print("Condition number for normalized X1:",la.cond(X1_normalized))

condition number for X1: 16.94216567652701
Condition number for normalized X1: 11.935084795944075


In [10]:
U, s, vh = np.linalg.svd(X1_normalized, full_matrices=True)
S = np.diag(s)

In [15]:
#taking the transpose of U, V and taking the inverse of S
UT = U.T 
V = vh.T  
arr = np.zeros((2,2515))
ST = np.concatenate((la.inv(S),arr), axis =1)
Atilde = UT @ X2_normalized @ V @ ST.T 

Atilde

array([[ 9.99914082e-01,  1.14769110e-03],
       [-3.05521254e-05,  9.96654833e-01]])

In [16]:
values,vector = la.eig(Atilde)
print(values, vector)

[0.99990329 0.99666563] [[ 0.99995577 -0.33312405]
 [-0.00940471  0.94288301]]


In [18]:
phi = X2_normalized @ V @ ST.T @ vector

phi

array([[-0.70035764, -0.42972703],
       [-0.71365662,  0.89926473]])